In [1]:
pip install numpy pandas

  Using cached pandas-2.2.1-cp312-cp312-macosx_11_0_arm64.whl.metadata (19 kB)
  Using cached pytz-2024.1-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2024.1-py2.py3-none-any.whl.metadata (1.4 kB)
Using cached pandas-2.2.1-cp312-cp312-macosx_11_0_arm64.whl (11.3 MB)
Using cached pytz-2024.1-py2.py3-none-any.whl (505 kB)
Using cached tzdata-2024.1-py2.py3-none-any.whl (345 kB)
Note: you may need to restart the kernel to use updated packages.


In [2]:
#importation des librairies
import os
import cv2
import numpy as np
import json
import pandas as pd

In [6]:
# Définition de la fonction pour convertir un fichier JSON en masque
def json_to_mask(json_file):
    with open(json_file, 'r') as f:
        data = json.load(f)

    mask = np.zeros((data['imageHeight'], data['imageWidth']), dtype=np.uint8)

    for shape in data['shapes']:
        label = shape['label']
        points = shape['points']
        polygon = np.array(points, dtype=np.int32)

        if label == 'numeroex':
            cv2.fillPoly(mask, [polygon], 128)
        elif label == 'Enonce':
            cv2.fillPoly(mask, [polygon], 255)
        elif label == 'tab':
            cv2.fillPoly(mask, [polygon], 64)
        elif label == 'img':
            cv2.fillPoly(mask, [polygon], 192)

    return mask

# Chemins vers les répertoires
quentin_JSON = 'qlandon/APP_IA_TL_QL_dev/data/label'
quentin_BRUT = 'qlandon/APP_IA_TL_QL_dev/data/Image'
quentin_mask = 'qlandon/APP_IA_TL_QL_dev/data/mask'

theo_JSON = '/Users/theolambs/APP_IA_TL_QL_dev/data/label'
theo_BRUT = '/Users/theolambs/APP_IA_TL_QL_dev/data/Image'
theo_mask = '/Users/theolambs/APP_IA_TL_QL_dev/data/mask'

utilisateur = int(input("si quentin tape 0, si theo tape 1: "))

if utilisateur == 0:
    json_dir = quentin_JSON
    img_dir = quentin_BRUT
    mask_dir = quentin_mask
else:
    json_dir = theo_JSON
    img_dir = theo_BRUT
    mask_dir = theo_mask

# Assurez-vous que le répertoire pour les masques existe, sinon ça le crée
if not os.path.exists(mask_dir):
    os.makedirs(mask_dir)

# Liste pour stocker les informations sur chaque masque
data_info = []  # Initialisation de la liste des informations à inclure dans le DataFrame
index_counter = 1  # Initialisation du compteur d'index

# Parcours de tous les fichiers JSON dans le répertoire
for json_file in os.listdir(json_dir):
    if json_file.startswith('LEnonce') and json_file.endswith('.json'):
        image_number = json_file[7:-5]  # Supprime "LEnonce" du début et ".json" de la fin
        img_filename = f'Enonce{image_number}.png'
        img_path = os.path.join(img_dir, img_filename)

        if os.path.exists(img_path):
            mask = json_to_mask(os.path.join(json_dir, json_file))
            mask_filename = 'M' + os.path.splitext(json_file)[0] + '.png'
            mask_file_path = os.path.join(mask_dir, mask_filename)
            cv2.imwrite(mask_file_path, mask)

            if os.path.exists(mask_file_path):
                print(f"Masque enregistré avec succès : {mask_file_path}")
                data_info.append([index_counter, img_path, mask_file_path])#enregistrement dans la liste data_info (num, pathimg, pathmask)
                index_counter += 1  # Incrémentation du compteur d'index
            else:
                print(f"Erreur lors de l'enregistrement du masque : {mask_file_path}")
        else:
            print(f"Image associée non trouvée pour : {json_file}")

# Création du DataFrame à partir des informations collectées liste data_info + titre colonne
df = pd.DataFrame(data_info, columns=['Index', 'pathimg', 'pathmask'])

# Enregistrement du DataFrame au format CSV
df.to_csv('informations_masques.csv', index=False)

# Affichage du DataFrame
print(df)


OSError: [Errno 30] Read-only file system: 'qlandon'